# Semantic shifts

In [1]:
from ipynb.fs.full.koselleck import *

[Koselleck] (22:28:29) Alles bereit (+0.0s)


## Generating data

### Local neighborhood measure

In [2]:
nbr_word('virtue').loc['virtue']

[Koselleck] (22:28:29) Running nbr_word(virtue) (+0.0s)
[Koselleck] (22:28:29) Finished running nbr_word(virtue) (+0.0s)


count     cdist  rank
period    neighbor                            
1720-1725 authority         10  0.921875     1
          integrity         10  1.004337     2
          popish            10  1.055988     3
          derogatory        10  1.063466     4
          mankind           10  1.076029     5
...                        ...       ...   ...
1895-1900 encouragement      4  1.162844   996
          lofty              4  1.163750   997
          warfare            4  1.163813   998
          pose               4  1.167160   999
          perceive           4  1.167461  1000

[36000 rows x 3 columns]

In [3]:

def lnm_neighborhoods(dfnbr1,dfnbr2,dfcdist1,dfcdist2,k=K,run_all_pairs=False,keep_runs=False,**attrs):
    # filter words in both models
    valid_words_now=list(set(dfnbr1.index) & set(dfnbr2.index) & set(dfcdist1.columns) & set(dfcdist2.columns))
    dfnbr1=dfnbr1.loc[valid_words_now].dropna().sort_values('rank').iloc[:k]
    dfnbr2=dfnbr2.loc[valid_words_now].dropna().sort_values('rank').iloc[:k]
    dfmetanbr=dfnbr1.append(dfnbr2)
    metaneighb=list(set(dfnbr1.index) | set(dfnbr2.index))
    dfcdist1=dfcdist1[metaneighb].dropna()
    dfcdist2=dfcdist2[metaneighb].dropna()
    runs=set(dfcdist1.index) & set(dfcdist2.index)
    o=[]
    for run1 in runs:
        for run2 in runs:
            if not run_all_pairs and run1!=run2: continue
            if run_all_pairs and run1>run2: continue
                
            dists1=dfcdist1.loc[run1]
            dists2=dfcdist2.loc[run2]
            try:
                distdists = 1-fastdist.cosine(dists1.values.astype(float), dists2.values.astype(float))
            except ZeroDivisionError as e:
#                 display(dists1)
#                 display(dists2)
                print('!!',e)
#                 stop
                continue
            odx={
                **attrs,
                'run1':run1,
                'run2':run2,
                'lnm':distdists,
                'mneighb_size':len(metaneighb),
                'neighb1_size':len(dfnbr1),
                'neighb2_size':len(dfnbr2),
            }
            o.append(odx)
    odf=pd.DataFrame(o)
    if keep_runs or not len(o): return odf
    odf=odf.groupby(list(attrs.keys())).mean().drop(['run1','run2'],1)#.mean()
    return odf

In [4]:
def _lnm_neighborhoods_(objd): return lnm_neighborhoods(**objd)


In [5]:
def lnm_word(
        word,
        period_or_periods=None,
        run_or_runs=None,
        word2=None,
        cache=True,
        force=False,
        k=K,
        cache_only=False,
        progress=True,
        progress_nbr=False,
        num_proc=1):
    word1 = word
    if not word2: word2 = word1
    qstr=f'{word1},{word2},ymin={YMIN},ymax={YMAX},ybin={YEARBIN},k={k}'
    if cache and not force:
        with get_veclib('lnm') as vl:
            if qstr in vl: return vl[qstr] if not cache_only else pd.DataFrame()
    
    try:
        dfnbr1 = nbr_word(word1,period_or_periods,run_or_runs,progress=progress_nbr,force=force).loc[word1]
        dfnbr2 = dfnbr1 if word2==word1 else nbr_word(word2,period_or_periods,run_or_runs,progress=progress_nbr,force=force).loc[word2]
        dfcdist1 = cdist_word(word1,period_or_periods,run_or_runs,progress=progress_nbr).loc[word1]
        dfcdist2 = dfcdist1 if word2==word1 else cdist_word(word2,period_or_periods,run_or_runs,progress=progress_nbr).loc[word2]
    except KeyError:
        return pd.DataFrame()
    
            
    periods=sorted(list(
        set(dfnbr1.index.get_level_values('period')) | set(dfnbr2.index.get_level_values('period'))
    ))
    objs = [
        dict(
            word1=word1,word2=word2,
            period1=prd1,period2=prd2,
            dfnbr1=dfnbr1.loc[prd1],
            dfnbr2=dfnbr2.loc[prd2],
            dfcdist1=dfcdist1.loc[prd1],
            dfcdist2=dfcdist2.loc[prd2],
            k=k
        )
        for prd1 in periods
        for prd2 in periods
        if prd1<prd2
    ]
    if progress: print(f'# of objects: {len(objs)}')
    o=pmap(
        _lnm_neighborhoods_,
        objs,
        num_proc=num_proc,
        progress=progress,
        desc='Measuring LNM across period comparisons'
    )
    odf=pd.concat(o) if len(o) else pd.DataFrame()
    if cache:
        with get_veclib('lnm',autocommit=True) as vl:
            vl[qstr]=odf
    
    return odf if not cache_only else pd.DataFrame()

In [6]:
# odf=lnm_word('station',num_proc=1,cache_only=False,force=True,progress_nbr=True)
# odf

## lnm()

In [7]:
def _lnm_(objd): return lnm_word(**objd)

def lnm(
        word_or_words,
        period_or_periods=None,
        run_or_runs=None,
        cache=True,
        force=False,
        cache_only=False,
        progress=True,
        progress_nbr=False,
        progress_word=None,
        num_proc=1,
        k=K
        ):
    words=tokenize_fast(word_or_words) if type(word_or_words)==str else list(word_or_words)
    
    objs=[
        dict(
            word=word,
            period_or_periods=period_or_periods,
            run_or_runs=run_or_runs,
            word2=None,
            cache=cache,
            force=force,
            k=k,
            cache_only=cache_only,
            progress=progress_word if progress_word is not None else (False if len(words)>1 else progress),
            progress_nbr=progress_nbr,
            num_proc=1 if len(words)>1 else num_proc,
        ) for word in words
    ]
    o=pmap(
        _lnm_,
        objs,
        num_proc=num_proc if len(words)>1 else 1,
        progress=progress if len(words)>1 else False,
        desc='Measuring LNM across words',
    )
    return pd.concat(o) if len(o) else pd.DataFrame()


In [8]:
# lnm('virtue,vice,virtues,vices,values')

In [9]:
def lnm_precache_words(words=None,**y):
    words_done=set()
    with get_veclib('lnm') as vl:
        words_done=set(k.split(',')[0] for k in vl.keys())
    if not words: words=get_valid_words()
    words=[w for w in words if not w in words_done]
    lnm(words, cache_only=True,**y)

In [10]:
# words=get_valid_words()
# random.shuffle(words)
# res=pmap(do_word, words, num_proc=1)

In [11]:
lnm_precache_words(get_all_nouns_adjs(), num_proc=3)

Measuring LNM across words [x3]:  25%|██▍       | 996/4011 [2:16:41<7:16:23,  8.68s/it] [Koselleck] (00:45:32) !! division by zero (+8222.1s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero (+0.0s)
[Koselleck] (00:45:32) !! division by zero

## Importing old data (to save time)

In [12]:
# gcols=['word1','word2','period1','period2']
# icols=gcols+['run1','run2']
# valcols=['lnm','mneighb_size','neighb1_size','neighb2_size']

# def import_into_shift_db(ifn=FN_ALL_LOCALDISTS_ORIGDATA,k=K):
#     print('Reformatting old data')
#     DF=read_df(ifn).query(f'k=={K}').reset_index()
#     rund=dict((f'run_{i:02}',i) for i in range(1,11))
#     DF.run1=DF.run1.replace(rund)
#     DF.run2=DF.run2.replace(rund)
#     df_done=DF
#     df_done['word2']=df_done.word
#     df_done=df_done.rename({'word':'word1','metaneighb_size':'mneighb_size', 'dist_local':'lnm'},axis=1)
#     df_done['mneighb_size']=np.nan
#     df_done['neighb1_size']=np.nan
#     df_done['neighb2_size']=np.nan
#     print('Done')
#     display(df_done)
#     pmap_groups(
#         do_add_db,
#         df_done.groupby(['word1','word2']),
#         num_proc=1,
#         desc='Importing...'
#     )

# def do_add_db(dfgrp):
#     word1,word2=dfgrp.iloc[0].word1, dfgrp.iloc[0].word2
#     wqstr=f'{word1},{word2},ybin={YEARBIN},ymin={YMIN},ymax={YMAX},k={K}'
#     with get_veclib('lnm',autocommit=True) as vl:
#         odf=dfgrp.set_index(icols)[valcols]
#         vl[wqstr]=odf
#         vl.commit()
#     return pd.DataFrame([{'success':True}])


In [13]:
# import_into_shift_db(DF)